In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.densenet import DenseNet201,preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/ByFamilyPortexPE"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:         allaple	Number of images: 362
Label: 1	Family:           alman	Number of images: 325
Label: 2	Family:          autoit	Number of images: 261
Label: 3	Family:            daws	Number of images: 466
Label: 4	Family:            delf	Number of images: 359
Label: 5	Family:         gamarue	Number of images: 259
Label: 6	Family:          ibryte	Number of images: 347
Label: 7	Family:          loring	Number of images: 285
Label: 8	Family:          mydoom	Number of images: 578
Label: 9	Family:          qukart	Number of images: 253
Label:10	Family:          ramnit	Number of images: 506
Label:11	Family:          sality	Number of images: 1401
Label:12	Family:          simbot	Number of images: 1148
Label:13	Family:       softpulse	Number of images: 912
Label:14	Family:          viking	Number of images: 183
Label:15	Family:         virlock	Number of images: 373
Label:16	Family:          vobfus	Number of images: 405
Label:17	Family:          wapomi	Number of images: 345
Label:18

In [4]:
X.shape

(10136, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = DenseNet201(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(10136, 1, 1, 1920)

In [8]:
bnfeatures = np.reshape(bnfeatures,(bnfeatures.shape[0],-1))

In [9]:
bnfeatures.shape

(10136, 1920)

In [10]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 10  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [11]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.4min


[CV] ........................... C=0.01, score=0.944223, total= 1.3min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.938356, total= 1.4min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.925595, total= 1.4min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.923687, total= 1.4min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.928150, total= 1.5min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.915925, total= 1.5min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.926441, total= 1.5min
[CV] C=10.0 ..........................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  4.7min


[CV] ............................ C=0.1, score=0.921860, total= 4.7min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.943249, total= 4.7min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.934055, total= 4.8min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.921705, total= 4.8min
[CV] C=100.0 .........................................................
[CV] ............................ C=1.0, score=0.912109, total= 5.1min
[CV] C=100.0 .........................................................
[CV] ............................ C=1.0, score=0.909449, total= 5.1min
[CV] C=100.0 .........................................................
[CV] ............................ C=1.0, score=0.921569, total= 5.2min
[CV] C=100.0 .........................................................
[CV] .

...

[CV] ................... C=1000000000.0, score=0.920588, total= 8.7min
[CV] ................... C=1000000000.0, score=0.937378, total= 8.8min
[CV] ................... C=1000000000.0, score=0.909449, total= 8.3min
[CV] ................... C=1000000000.0, score=0.921260, total= 8.5min
[CV] ................... C=1000000000.0, score=0.905045, total= 8.4min
[CV] ................... C=1000000000.0, score=0.928287, total= 7.0min
[CV] ................... C=1000000000.0, score=0.912785, total= 8.2min
[CV] .................. C=10000000000.0, score=0.920276, total= 6.2min
[CV] ................... C=1000000000.0, score=0.910714, total= 8.3min
[CV] ................... C=1000000000.0, score=0.916501, total= 8.8min
[CV] .................. C=10000000000.0, score=0.907227, total= 8.7min
[CV] .................. C=10000000000.0, score=0.909449, total= 8.6min
[CV] .................. C=10000000000.0, score=0.935421, total= 8.8min
[CV] .................. C=10000000000.0, score=0.912785, total= 7.1min


[Parallel(n_jobs=-1)]: Done 125 out of 130 | elapsed: 33.2min remaining:  1.3min


[CV] .................. C=10000000000.0, score=0.915686, total= 8.8min
[CV] .................. C=10000000000.0, score=0.907023, total= 8.1min
[CV] .................. C=10000000000.0, score=0.914513, total= 7.4min
[CV] .................. C=10000000000.0, score=0.908730, total= 7.8min
[CV] .................. C=10000000000.0, score=0.927291, total= 7.6min


[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed: 34.6min finished


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [12]:
top_model.cv_results_

{'mean_fit_time': array([ 84.98458488, 294.94617066, 290.37239635, 287.65498657,
        293.14801145, 283.59665172, 265.86135368, 315.69245293,
        402.51596093, 497.46027396, 524.09533846, 501.32045734,
        474.52540824]),
 'mean_score_time': array([0.04415133, 0.04074051, 0.03577251, 0.03042703, 0.03123448,
        0.02847621, 0.02871468, 0.0419956 , 0.06069024, 0.05067651,
        0.06526096, 0.04369974, 0.04395022]),
 'mean_test_score': array([0.92452644, 0.92867009, 0.92018548, 0.91673244, 0.91623915,
        0.91535122, 0.91574586, 0.91535122, 0.91623915, 0.91535122,
        0.91505525, 0.91712707, 0.91584451]),
 'mean_train_score': array([0.96738798, 0.99445311, 0.99978077, 0.99969303, 0.99969309,
        0.99970403, 0.99969305, 0.99968214, 0.99963832, 0.99971501,
        0.99963833, 0.99970405, 0.99967116]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
      

In [13]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.92453, std: 0.01081, params: {'C': 0.01},
 mean: 0.92867, std: 0.00895, params: {'C': 0.1},
 mean: 0.92019, std: 0.00910, params: {'C': 1.0},
 mean: 0.91673, std: 0.00895, params: {'C': 10.0},
 mean: 0.91624, std: 0.01019, params: {'C': 100.0},
 mean: 0.91535, std: 0.00992, params: {'C': 1000.0},
 mean: 0.91575, std: 0.00841, params: {'C': 10000.0},
 mean: 0.91535, std: 0.00890, params: {'C': 100000.0},
 mean: 0.91624, std: 0.00919, params: {'C': 1000000.0},
 mean: 0.91535, std: 0.00932, params: {'C': 10000000.0},
 mean: 0.91506, std: 0.00911, params: {'C': 100000000.0},
 mean: 0.91713, std: 0.00945, params: {'C': 1000000000.0},
 mean: 0.91584, std: 0.00888, params: {'C': 10000000000.0}]

In [14]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.1} - score: 0.9287
